In [1]:
import torch.nn as nn
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch

batch_size = 12

train_data = datasets.MNIST('MNIST_data/', train = True, download= True, transform=transforms.ToTensor())
test_data = datasets.MNIST('MNIST_data/', train = False, download= True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle=True)
PATH='model.pt'

class RobustModel(nn.Module):
    """
    TODO: Implement your model
    """
    def __init__(self):
        super(RobustModel, self).__init__()

        self.in_dim = 28 * 28 * 3
        self.out_dim = 10

        self.fc1 = nn.Linear(self.in_dim, 512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(128,64)
        self.fc5 = nn.Linear(64,self.out_dim)
        
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax()
        
        self.linear = nn.Linear(self.in_dim, self.out_dim)

    def forward(self, x):
        x = x.view(-1, self.in_dim)
        x = self.linear(x)
        a1 = self.relu(self.fc1(x))
        a2 = self.relu(self.fc2(a1))
        a3 = self.relu(self.fc3(a2))
        a4 = self.relu(self.fc4(a3))
        logit = self.fc5(a4)
        return logit
        
        
model = RobustModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs , labels = data
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (i+1) % 2000 == 0:
            print('[%d, %5d] loss : %.3f' %
                  (epoch +1, i+1, running_loss / 2000))
            running_loss = 0.0
    torch.save(model.state_dict(), PATH)
print('Finished Training')

/Users/ibyeong-gwon/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/ibyeong-gwon/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at11TensorMaker11make_tensorEv
  Referenced from: /Users/ibyeong-gwon/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in: /Users/ibyeong-gwon/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


ModuleNotFoundError: No module named 'torch.ao'